## THINGS TO DO
 1. TF-IDF
 2. DOC2VEC
 3. TOPIC MODELING

In [18]:
import pickle
import numpy as np
import sys
import gensim
import os
import collections
import smart_open
import random

from IPython.display import clear_output


In [ ]:
# READ LYRIC PICKLE
scraped_lyric_dir = "./lyrics/song_artist_lyrics.p"
song_pair = pickle.load(open(scraped_lyric_dir, 'rb'))

### DOC2VEC

In [20]:
def progress(cnt, total, div):
    if cnt % div == 0:
        clear_output(wait=True)
        print ("Preprocessing... ", cnt / total*1.0 * 100, '%')
    
def read_corpus(song_pair, tokens_only=False):
    for i, key in enumerate(song_pair):
        progress(i, len(song_pair), 10000)
        artist = song_pair[key][1].replace('\n', '')
        title = song_pair[key][0].replace('\n', '')
        lyric = song_pair[key][2].replace('\n', ' ')
        if tokens_only:
            yield gensim.utils.simple_preprocess(lyric)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(lyric), [artist + '_' + title])

corpus = list(read_corpus(song_pair))
print ("Finished")
# print (corpus[:2])

Preprocessing...  93.97436379355712 %
Finished


In [45]:
# TRAINING...
epoch = 5 # Set to 2 because slow
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=epoch)
model.build_vocab(corpus)
%time model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 2min 5s, sys: 3.48 s, total: 2min 9s
Wall time: 50.3 s


In [54]:
print (model)
upto = 10
song_idx = 5
print ("Key:" , corpus[:10][song_idx][1][0])
print ("Sample Doc Embedding:\n", model.docvecs[corpus[:upto][song_idx][1][0]])
print (model.docvecs[corpus[:upto][song_idx][1][0]].shape)

Doc2Vec(dm/m,d100,n5,w5,mc2,s0.001,t3)
Key: Usher_Yeah!
Sample Doc Embedding:
 [ 4.21131790e-01 -5.73277660e-02 -4.81016599e-02 -4.05346990e-01
 -1.66537449e-01 -7.88057894e-02 -1.25835955e-01  3.73831779e-01
 -2.66123533e-01  4.21134025e-01  4.18611094e-02 -5.71092963e-03
  1.95088416e-01 -3.12872559e-01 -1.44553989e-01 -2.37367064e-01
 -1.16937488e-01  3.98216635e-01 -3.81563336e-01  1.12749144e-01
  2.19683319e-01  4.11753245e-02  3.19703281e-01  2.36891359e-01
  2.93331295e-01  3.74861270e-01  8.59662145e-02  3.62854630e-01
  1.25365004e-01  3.76454920e-01 -5.68959951e-01 -2.85776049e-01
  1.95158511e-01  2.36612722e-01 -7.91151896e-02 -1.31897762e-01
  3.73833179e-01  2.12186471e-01  4.97275025e-01 -2.37590060e-01
 -4.18672681e-01  1.52167529e-01  1.51995331e-01  6.71629086e-02
  4.38404828e-01 -6.88874512e-04  3.90874833e-01  3.44782770e-01
  1.89395063e-02  3.35040331e-01 -5.33962727e-01  1.35052294e-01
 -4.51061130e-01  1.34803101e-01 -1.36076212e-01 -1.78043872e-01
 -6.3164591

In [55]:
sims = model.docvecs.most_similar([model.docvecs[corpus[:upto][song_idx][1][0]]])
print ("SONG SIMILILIARY COMPARED TO BRITNEY SPEARS-TOXIC\n")
for idx, song in enumerate(sims):
    print (idx, ':', sims[idx][0].replace('_', " -> "))

SONG SIMILILIARY COMPARED TO BRITNEY SPEARS-TOXIC

0 : Usher -> Yeah!
1 : Glee Cast -> Yeah! (Glee Cast Version)
2 : Travis Porter -> Ayy Ladies
3 : Travis Porter -> Bring It Back
4 : Ludacris -> Freaky Thangs
5 : Jason Derulo -> Zipper
6 : Project Pat -> Sucks on Dick (feat. Juicy J)
7 : Tyga -> Bouncin On My D*ck
8 : Macklemore -> Willy Wonka (feat. Offset)
9 : Lil Wayne -> Weezy Baby
